# Linear Regression
will test OLS and SGD using random data sets, same seed for consetenticy 


In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Parameters / assumptions
n_samples = 2000
seed = 42
rng = np.random.RandomState(seed)
noise_std = 3.0 # Standard deviation of the Gaussian noise

# Generate 1D feature X in range [-10, 10]
X = rng.uniform(-10.0, 10.0, size=(n_samples, 1))

# True linear model parameters
true_w = 3.0    # slope
true_b = 5.0    # intercept

# Targets with Gaussian noise, such that y = Mx + b + epsilon(noise)
noise = rng.normal(0.0, noise_std, size=(n_samples, 1))
y = true_w * X + true_b + noise
y = y.ravel()

# Quick visualization of the generated data
print("X shape:", X.shape, "y shape:", y.shape)
plt.figure(figsize=(7,4))
plt.scatter(X, y, s=8, alpha=0.6)
plt.xlabel("X")
plt.ylabel("y")
plt.title(f"Generated data with noise (n={n_samples})")
plt.grid(alpha=0.3)
plt.show()